In [2]:
!pip install dtaidistance

     |████████████████████████████████| 797 kB 15.6 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dtaidistance: filename=dtaidistance-2.3.6-cp36-cp36m-linux_x86_64.whl size=888972 sha256=8fd93b8e6b488723d9749ca5d32915fbeb296fcd30c346333e8ca69472e121b5
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/5f/18/3db8e0572e06cde30543380d60b4511e3802d1481c3a3a2642
Successfully built dtaidistance


In [3]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from collections import defaultdict
from tqdm import tqdm 
from dtaidistance import dtw

tqdm.pandas()

In [4]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [5]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,source,education,company_name,function_id,isco_code4
0,84556,0,37,156,2.0,0,0.0,324258,936,208
1,84556,1,23,116,1.0,0,0.0,324258,809,348
2,84556,2,23,275,2.0,0,0.0,329244,936,208
3,84556,3,1155,4,1.0,0,0.0,368140,1519,344
4,84556,4,203,11,1.0,0,0.0,419826,1519,344


In [6]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [7]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:12<00:00, 36386.11it/s]


In [9]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [10]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [20]:
results = defaultdict(lambda: defaultdict(int))

try:
    for i, current in tqdm(enumerate(career_paths.iteritems()), total=len(career_paths)):
        candidate, career = current
        career = career[:-1].astype("double")

        for j in career_paths.iteritems():
            candidate2, career2 = j

            if candidate != candidate2:
                results[candidate][candidate2] = dtw.distance(career, 
                                                              career2.astype("double"), 
                                                              use_c=True, 
                                                              use_pruning=True)
                
        if i == 1000:
            break
                        
except KeyboardInterrupt:
    pass

  1%|          | 1000/113724 [18:13<34:13:59,  1.09s/it]


In [21]:
best_matches = {}

for i in tqdm(results):
    if len(results[i]):
        best_matches[i] = sorted(results[i], key=lambda x: results[i][x])

100%|██████████| 1001/1001 [01:03<00:00, 15.70it/s]


In [22]:
def knn_classifier(best_matches, career_paths, k=1):
    
    predictions = []
    
    for candidate in best_matches.keys():
        correct = career_paths[candidate][-1]
                
        neighbors = [(career_paths[neighbor][-1], i + 1) for i, neighbor in enumerate(best_matches[candidate][:k])]
        
        weights = defaultdict(int)
        
        for job, weight in neighbors:
            weights[job] += 1/np.log(1 + weight)
            
        predictions.append(correct == max(weights, key=lambda x: weights[x]))
        
    return np.mean(predictions)

In [23]:
results = defaultdict(list)

for k in tqdm(range(1, 20)):
    results["k"].append(k)
    results["accuracy"].append(knn_classifier(best_matches, career_paths, k=k))
    
pd.DataFrame(results).set_index("k")

100%|██████████| 19/19 [00:02<00:00,  9.14it/s]


,accuracy
k,
1,0.061938
2,0.061938
3,0.064935
4,0.065934
5,0.069930
6,0.071928
7,0.073926
8,0.075924
9,0.078921
